In [4]:
# this code is functionally identical to the first API search, except the location and review data are concatenated at the end
# we performed a second search with a second API key in the .env with different review criteria in an attempt to gather more data
# we ended up marginally increasing our review count from 290 to 312, less than we would have hoped, but more data nonetheless
import pandas as pd

df1 = pd.read_csv('locations_df.csv')
df1 = df1.drop_duplicates(subset='place_id', keep='first')

In [6]:
from dotenv import load_dotenv
import os

# we have to use a second API key to perform new searches
load_dotenv('apikey.env', override = True)
api_key2 = os.getenv('SERPAPI_KEY2')

In [8]:
import serpapi

locations = []
# with a new key, we can perform a fresh batch of 100 searches
for i in range (0, 41, 20):
    client = serpapi.Client(api_key=api_key2)
    locs = client.search({
        'engine': 'google_maps',
        'type': 'search',
        'q': 'places to study',
        'll': '@34.07100342918671,-118.4448019289779,15z',
        'start': i
    })

    locs_dict = locs.as_dict()
    if 'local_results' in locs_dict:
        locs_df = pd.json_normalize(locs_dict, 'local_results')
        locations.append(locs_df)
# compile location data into dataframe        
locations_df2 = pd.concat(locations, ignore_index = True)
# include only necessary columns
loc_cols = ['title', 'place_id', 'rating', 'reviews', 'type', 'address', 'gps_coordinates.latitude', 'gps_coordinates.longitude',
            'operating_hours.monday', 'operating_hours.tuesday', 'operating_hours.wednesday', 'operating_hours.thursday',
            'operating_hours.friday', 'operating_hours.saturday', 'operating_hours.sunday']
locations_df2 = locations_df2[loc_cols]
locations_df2

,title,place_id,rating,reviews,type,address,gps_coordinates.latitude,gps_coordinates.longitude,operating_hours.monday,operating_hours.tuesday,operating_hours.wednesday,operating_hours.thursday,operating_hours.friday,operating_hours.saturday,operating_hours.sunday
0,The Study,ChIJHdkHEpK8woARBj3RptYbUII,4.5,145,Cafe,"250 De Neve Dr, Los Angeles, CA 90024",34.073214,-118.452151,"7–10 AM, 11 AM–3 PM, 5 PM–12 AM","7–10 AM, 11 AM–3 PM, 5 PM–12 AM","7–10 AM, 11 AM–3 PM, 5 PM–12 AM","7–10 AM, 11 AM–3 PM, 5 PM–12 AM","7–10 AM, 11 AM–3 PM, 5 PM–12 AM","7–10 AM, 11 AM–3 PM, 5 PM–12 AM","7–10 AM, 11 AM–3 PM, 5 PM–12 AM"
1,Kerckhoff Coffee House,ChIJLRWe7Yi8woARWSwZigJvSAE,4.3,192,Coffee shop,"308 Westwood Plaza #2, Los Angeles, CA 90095",34.070591,-118.443415,7 AM–10 PM,7 AM–10 PM,7 AM–10 PM,7 AM–10 PM,7 AM–7 PM,8 AM–6 PM,8 AM–8 PM
2,Starbucks,ChIJO93ZjYO8woAR_RWA_NmpGmE,4.0,795,Coffee shop,"10955 Weyburn Ave, Los Angeles, CA 90024",34.062529,-118.447346,4:30 AM–10 PM,4:30 AM–10 PM,4:30 AM–10 PM,4:30 AM–10 PM,4:30 AM–10 PM,4:30 AM–10 PM,4:30 AM–10 PM
3,Jimmy's Coffeehouse,ChIJG10_cWK8woAR_JGnjUugKH4,3.7,50,Coffee shop,"Lu Valle Commons, 398 Portola Plaza, Los Angel...",34.073698,-118.439381,7 AM–9 PM,7 AM–9 PM,7 AM–9 PM,7 AM–9 PM,7 AM–5 PM,8 AM–5 PM,10 AM–4 PM
4,Elysee Bakery & Cafe,ChIJ5RweMYK8woAR1YeGx8PLuzo,4.3,796,Coffee shop,"1099 Gayley Ave, Los Angeles, CA 90024",34.060231,-118.446844,7 AM–10 PM,7 AM–10 PM,7 AM–10 PM,7 AM–10 PM,7 AM–10 PM,7 AM–10 PM,7 AM–10 PM
5,Starbucks,ChIJcdLqOIG8woARjSHXdbuFFcw,3.5,53,Coffee shop,"10861 Weyburn Ave, Los Angeles, CA 90024",34.062686,-118.444064,7 AM–9 PM,7 AM–9 PM,7 AM–9 PM,7 AM–9 PM,7 AM–9 PM,7 AM–9 PM,7 AM–9 PM
6,Junbi - Westwood,ChIJd8Lnj4O8woARE33N-eWQSw0,4.2,267,Bubble tea store,"10967 Weyburn Ave, Los Angeles, CA 90024",34.062369,-118.447638,11:30 AM–9 PM,11:30 AM–9 PM,11:30 AM–9 PM,11:30 AM–9 PM,11:30 AM–9 PM,11:30 AM–9 PM,11:30 AM–9 PM
7,Upside Down,ChIJx-YmSLC9woARGa_M8BAejdg,4.8,116,Coffee shop,"10962 Le Conte Ave, Los Angeles, CA 90024",34.063422,-118.447501,9 AM–3 PM,9 AM–3 PM,9 AM–3 PM,9 AM–3 PM,9 AM–1 PM,Closed,Closed
8,Ministry of Coffee,ChIJ18pBDYG8woAR1OMcBik4g6M,4.2,386,Coffee shop,"1010 Glendon Ave, Los Angeles, CA 90024",34.061902,-118.443970,7 AM–7 PM,7 AM–7 PM,7 AM–7 PM,7 AM–7 PM,7 AM–7 PM,7 AM–7 PM,7 AM–7 PM
9,Bruin Cafe,ChIJezN24o28woARcqiE5XHiRhc,4.3,68,Cafe,"350 De Neve Dr, Los Angeles, CA 90024",34.072607,-118.450332,"11 AM–2 PM, 5–9 PM","11 AM–2 PM, 5–9 PM","11 AM–2 PM, 5–9 PM","11 AM–2 PM, 5–9 PM","11 AM–2 PM, 5–9 PM",Closed,Closed


In [14]:
# at this point in the project, we only want to add data to our existing locations, not add more locations with sparse data
# we can accomplish this by filtering locations and reviews by shared ids (more below)

shared_ids = pd.merge(df1, locations_df2, left_on='place_id', right_on='place_id')['place_id'].unique().tolist()
shared_ids

['ChIJHdkHEpK8woARBj3RptYbUII',
 'ChIJG1SbmYW9woARsQubO7QYEEA',
 'ChIJx-YmSLC9woARGa_M8BAejdg',
 'ChIJd8Lnj4O8woARE33N-eWQSw0',
 'ChIJkydzloG8woARi25SOkVMPx4',
 'ChIJ18pBDYG8woAR1OMcBik4g6M',
 'ChIJu18GY0m7woARJwBGwsa3N88',
 'ChIJLRWe7Yi8woARWSwZigJvSAE',
 'ChIJ1xoN2oi8woARXJ6Ahcp7GS8',
 'ChIJG10_cWK8woAR_JGnjUugKH4',
 'ChIJx755_Qu9woARKGmYHprkK0c',
 'ChIJ83Ob_V-7woARyB2vkPZjBEc',
 'ChIJO93ZjYO8woAR_RWA_NmpGmE',
 'ChIJAaKB4IO8woAR0m5Z1SnSsGY',
 'ChIJr7b9d6C8woAROzlkysmJzQE',
 'ChIJ_QIYqCm7woARDChddex9orI',
 'ChIJRxLQ84G8woARUFQlVf_5p_Y',
 'ChIJ5RweMYK8woAR1YeGx8PLuzo',
 'ChIJezN24o28woARcqiE5XHiRhc',
 'ChIJ99Yi5Nu9woARG62g6qTSbx8',
 'ChIJm-IBiom8woAR-CxVeBRQVIM',
 'ChIJRxZS_I28woARkbLYZ2ThMUk',
 'ChIJAxCHD6C8woARyGTn5MorDgM',
 'ChIJQ_kheoy7woARo3jjflGLgxM',
 'ChIJs2qyh6C8woARXCtfGeA9sBM']

In [42]:
# new review search with different sort_by parameter to scrape a greater number of new reviews
# only searched using shared ids
reviews = []
for id in shared_ids:
    place_reviews = client.search({
        'engine': 'google_maps_reviews',
        'type': 'search',
        'place_id': id,
        'sort_by': 'qualityScore'
    })
    place_reviews_dict = place_reviews.as_dict()['reviews']
    df = pd.json_normalize(place_reviews_dict)
    df['place_id'] = id
    reviews.append(df)

In [140]:
# concatenating 2nd location and review searches with columns matching previous search
reviews_df2 = pd.concat(reviews, ignore_index=True)
loc_revs2 = pd.merge(reviews_df2, locations_df2[['place_id', 'title']], on='place_id', how='left')

reviews_df = pd.read_csv('reviews_df.csv')
cols = loc_revs2.columns.intersection(reviews_df.columns)
loc_revs2 = loc_revs2[cols]

loc_revs2 = loc_revs2.set_index('place_id')
loc_revs2 = loc_revs2.reset_index()
loc_revs2

,place_id,iso_date,likes,extracted_snippet.original,details.food,details.service,details.atmosphere,title
0,ChIJHdkHEpK8woARBj3RptYbUII,2023-10-08T04:15:34Z,1,Everything is amazing here. Thank you for keep...,5.0,5.0,5.0,The Study
1,ChIJHdkHEpK8woARBj3RptYbUII,2021-10-29T02:54:44Z,1,Nice residence hall with a study hall called '...,NaN,NaN,NaN,The Study
2,ChIJHdkHEpK8woARBj3RptYbUII,2024-09-02T19:02:38Z,0,Best take out restaurant at ucla. Everything i...,5.0,5.0,5.0,The Study
3,ChIJHdkHEpK8woARBj3RptYbUII,2021-10-07T02:35:26Z,2,I don’t know what’s up with these portion size...,NaN,NaN,NaN,The Study
4,ChIJHdkHEpK8woARBj3RptYbUII,2019-03-30T00:01:15Z,0,"Best place for food on campus, only balanced b...",NaN,NaN,NaN,The Study
...,...,...,...,...,...,...,...,...
203,ChIJs2qyh6C8woARXCtfGeA9sBM,2017-09-28T13:48:58Z,0,This is a pretty cool little place early in th...,NaN,NaN,NaN,Starbucks
204,ChIJs2qyh6C8woARXCtfGeA9sBM,2021-09-13T15:12:58Z,2,The staff are rude they don't want to supply e...,NaN,NaN,NaN,Starbucks
205,ChIJs2qyh6C8woARXCtfGeA9sBM,2024-08-28T16:51:06Z,0,The music has got to go. There is a speaker ri...,5.0,5.0,3.0,Starbucks
206,ChIJs2qyh6C8woARXCtfGeA9sBM,2024-01-09T20:24:03Z,1,The older lady who works here is absolutely ru...,3.0,1.0,1.0,Starbucks


In [77]:
loc_revs2.to_csv('loc_revs_search2.csv', header=True, index=False)

In [91]:
# from the second search, we only want to add non-duplicate reviews from the same locations as our original search
filtered_ids = loc_revs2[loc_revs2['place_id'].isin(reviews_df['place_id'])]

reviews_df_updated = pd.concat([reviews_df, filtered_ids], ignore_index=True)
reviews_df_updated = reviews_df_updated.drop_duplicates(subset=['extracted_snippet.original'])
reviews_df_updated = reviews_df_updated.set_index(['title', 'place_id'])
reviews_df_updated = reviews_df_updated.reset_index()
reviews_df_updated

,title,place_id,rating_x,reviews,type,address,gps_coordinates.latitude,gps_coordinates.longitude,operating_hours.monday,operating_hours.tuesday,...,operating_hours.friday,operating_hours.saturday,operating_hours.sunday,rating_y,iso_date,likes,extracted_snippet.original,details.service,details.food,details.atmosphere
0,Coffee Tomo,ChIJ-WkX2XS7woARkFl1H26Twh4,4.3,285.0,Coffee shop,"11309 Mississippi Ave, Los Angeles, CA 90025",34.03946,-118.442884,7:30 AM–6 PM,7:30 AM–6 PM,...,7:30 AM–6 PM,8 AM–6 PM,8 AM–6 PM,5.0,2023-12-26T02:21:04Z,0,Best best sweet potato cheese pretzel. Crunchy...,4,5.0,4.0
1,Coffee Tomo,ChIJ-WkX2XS7woARkFl1H26Twh4,4.3,285.0,Coffee shop,"11309 Mississippi Ave, Los Angeles, CA 90025",34.03946,-118.442884,7:30 AM–6 PM,7:30 AM–6 PM,...,7:30 AM–6 PM,8 AM–6 PM,8 AM–6 PM,5.0,2023-04-13T15:45:13Z,1,In A Nutshell:\n\n-Coffee Tomo located in Litt...,Dine in,5.0,NaN
2,Coffee Tomo,ChIJ-WkX2XS7woARkFl1H26Twh4,4.3,285.0,Coffee shop,"11309 Mississippi Ave, Los Angeles, CA 90025",34.03946,-118.442884,7:30 AM–6 PM,7:30 AM–6 PM,...,7:30 AM–6 PM,8 AM–6 PM,8 AM–6 PM,5.0,2024-07-18T21:45:01Z,0,Coffees and matchas made with excellence and c...,5,5.0,5.0
3,Coffee Tomo,ChIJ-WkX2XS7woARkFl1H26Twh4,4.3,285.0,Coffee shop,"11309 Mississippi Ave, Los Angeles, CA 90025",34.03946,-118.442884,7:30 AM–6 PM,7:30 AM–6 PM,...,7:30 AM–6 PM,8 AM–6 PM,8 AM–6 PM,2.0,2023-10-05T04:27:19Z,3,Went out of my way to try a local shop instead...,1,3.0,1.0
4,Coffee Tomo,ChIJ-WkX2XS7woARkFl1H26Twh4,4.3,285.0,Coffee shop,"11309 Mississippi Ave, Los Angeles, CA 90025",34.03946,-118.442884,7:30 AM–6 PM,7:30 AM–6 PM,...,7:30 AM–6 PM,8 AM–6 PM,8 AM–6 PM,5.0,2022-11-01T18:23:17Z,0,One of the best shots of espresso I've had in ...,Dine in,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,Anderson Cafe (Starbucks),ChIJ99Yi5Nu9woARG62g6qTSbx8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2024-11-07T17:19:59Z,0,I’ll be honest I come here frequently but the ...,1.0,1.0,1.0
309,Coral Tree Cafe,ChIJAxCHD6C8woARyGTn5MorDgM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2024-11-24T18:19:15Z,0,I got stuffed french toast and a panini and my...,1,1.0,1.0
310,Blue Bottle Coffee,ChIJQ_kheoy7woARo3jjflGLgxM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2024-11-19T22:03:09Z,0,I have had coffee all over the world and with ...,NaN,NaN,NaN
311,Blue Bottle Coffee,ChIJQ_kheoy7woARo3jjflGLgxM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2024-11-18T06:11:24Z,0,Ordered a hot Winter Nola. By far the second b...,5,5.0,5.0


In [101]:
reviews_df_updated.to_csv('reviews_df_updated.csv', header=True, index=False)